# UCSD ECE143 Final Project

## 1. Data Clean Up

In [8]:
import json
from collections import defaultdict

companies = defaultdict(int)
location = defaultdict(int)
skills = defaultdict(int)
industry = defaultdict(int)

with open('143dataall.json','r') as f:
    allpeople = f.readlines()
    
    for person in allpeople:
        info = json.loads(person)
        if info:

#         print(info["lastName"], info["firstName"])
            try:
                industry[info['industryName']] += 1
            except:
                print("industry")
                print(info)
                print()
                print()
                print()
                print()

            if info['experience']:
                companies[info['experience'][0]['companyName']] += 1

            try:
                curPlace = info['geoLocationName']
                city, state = curPlace.split(',')
                found = None #indicator of whether location has found in certain format
                for savedCity, savedState in location:
                    if city in savedCity:
                        found = True
                        location[(savedCity, savedState)] += 1

                if not found:
                    location[(city,state)] += 1
            except:
                print('geoloc')
                print(info)
                print()
                print()
                print()
                print()

            try:
                if info['skills']:
                    for skill in info['skills']:
                        if skill['name'].lower() == 'c' or skill['name'].lower() == 'c++' or skill['name'].lower() == 'c/c++' :
                            skills['c/c++'] += 1
                        else:
                            skills[skill['name'].lower()] += 1
            except:
                print('skill')
                print(info)
                print()
                print()
                print()
                print()
print(companies)
print(location)
print(industry)
print(skills)
        
            

geoloc
{'industryName': 'Computer Software', 'lastName': 'Wang', 'locationName': 'United States', 'student': False, 'geoCountryName': 'United States', 'geoCountryUrn': 'urn:li:fs_geo:103644278', 'geoLocationBackfilled': True, 'elt': False, 'industryUrn': 'urn:li:fs_industry:4', 'firstName': 'Yirui', 'entityUrn': 'urn:li:fs_profile:ACoAACiQY70B8Qj29NKQBjnGoTRrcQFAbwuxxjo', 'geoLocation': {'geoUrn': 'urn:li:fs_geo:103644278'}, 'location': {'preferredGeoPlace': 'urn:li:fs_city:(us,27-8)', 'basicLocation': {'countryCode': 'us'}}, 'headline': 'Seeking SDE intern opportunities in 2020 summer', 'displayPictureUrl': 'https://media.licdn.com/dms/image/C5103AQEoW27BfnkweQ/profile-displayphoto-shrink_', 'profile_id': 'ACoAACiQY70B8Qj29NKQBjnGoTRrcQFAbwuxxjo', 'experience': [{'locationName': 'Hangzhou, Zhejiang, China', 'entityUrn': 'urn:li:fs_position:(ACoAACiQY70B8Qj29NKQBjnGoTRrcQFAbwuxxjo,1496495174)', 'geoLocationName': 'Hangzhou, Zhejiang, China', 'geoUrn': 'urn:li:fs_geo:90009563', 'company

In [2]:
assert info

## 2. Data Visualization

### a. Necessary methods and constant

In [9]:
import json
from collections import defaultdict
import plotly
# import plotly.plotly as py
import plotly.offline as of
import plotly.graph_objs as go
import math
import addfips

# seperate the dict to be two lists
def seperateData(data):
    dictionary = dict(data)
    keys = dictionary.keys()
    values = dictionary.values()
    return list(keys), list(values)

# Set the top most skills we want to display
def getHighest(key, values, num):
    newKey = key[:num]
    newValue = values[:num]
    if num < len(key):
        newKey.append("Others")
        newValue.append(sum(values[num:]))
    return newKey, newValue

# Sort the key and values
def getSorted(key, values):
    tmpDict = dict(zip(key, values))
    tmpDict_inv = {str(v):k for k, v in tmpDict.items()}
    print(tmpDict_inv)
    tmp = values[:]
    tmp.sort(reverse=True)
    return [tmpDict_inv[str(i)] for i in tmp], tmp

# seperate the data for location information
def seperate_Loc_Data(data):
    dictionary = dict(data)
    keys = dictionary.keys()
    tmp = list(keys)
    values = dictionary.values()
    res = []
    for elem in keys:
        state = elem[1].strip()
        if state in us_state_abbrev:
            res.append(us_state_abbrev[state])
    return res, list(values)

def seperate_City_Data(data):
    dictionary = dict(data)
    keys = dictionary.keys()
    tmp = list(keys)
    values = dictionary.values()
    res = []
    for elem in keys:
        state = elem[1].strip()
        city = elem[0].strip()
        if state in us_state_abbrev:
            res.append(city)
    return res, list(values)

# count statistics data for loc and values
def coordinate_Loc_Val(loc, values):
    res = dict()
    for i in range(len(loc)):
        if loc[i] not in res: res[loc[i]] = values[i]
        else: res[loc[i]] += values[i]
    return list(res.keys()), [math.log(i+1) for i in list(res.values())]

def coordinate_City_Val(loc, city, values):
    res = dict()
    for i in range(len(loc)):
        if (loc[i],city[i]) not in res: res[(loc[i],city[i])] = values[i]
        else: res[(loc[i],city[i])] += values[i]
    return list(res.keys()), list(res.values())

def getFIPS_General(city_tuple, values): # not used as the addfips lib can not recognize some cities
    res = []
    af = addfips.AddFIPS()
    for i in range(len (values)):
        res.append(af.get_county_fips(city_tuple[i][1], state=city_tuple[0]))
    res, values = getSorted(res, values)
    return res


def getFIPS(city_tuple, values):
    res = []
    for i in range(len(values)):
        if city_tuple[i][1] == 'San Diego County':
            res.append('06073')
        elif city_tuple[i][1] == 'Sunnyvale':
            res.append('06085')
        elif city_tuple[i][1] == 'San Francisco':
            res.append('06075')
        elif city_tuple[i][1] == 'Seattle':
            res.append('63000')
        elif city_tuple[i][1] == 'Redmond':
            res.append('57535')
        elif city_tuple[i][1] == 'Palo Alto':
            res.append('06086') # should be the same with sunnyvale
        elif city_tuple[i][1] == 'DuPage County':
            res.append('17043')
        elif city_tuple[i][1] == 'Mountain View':
            res.append('06087') # should be the same with sunnyvale
    res, values = getSorted(res, values)
    return res

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

### a. Employees Skills --- Pie Plot

In [3]:

if __name__ == '__main__':
    
    #=============== Set up the data ===================#
    skill, number = seperateData(kills)
    number = [number[i]/sum(number) for i in range(len(number))] #convert number to percentage
    skill, number = getHighest(skill, number, 15)  # Only display the top NO.15 skills

    #===============Then Set up the figures ============#
    of.offline.init_notebook_mode(connected=True)
    trace = [go.Pie(
        labels = skill, 
        values = number, 
        hole =  0,                          # Modify the radius of the hole, higher means larger hole
        hoverinfo = "label + percent")]
    layout = go.Layout(title ="Skills UCSD alumni mastered", titlefont=dict(color='rgb(200, 100, 100)',size=30))
    data = [trace]
    fig = go.Figure(data = trace, layout = layout)
    of.iplot(fig)


### b. People in different companies  -- Bar Chart

In [4]:
if __name__ == '__main__':
    #=============== Set up the data ===================#
    company, number = seperateData(companies)
    company, number = getHighest(company, number, 5)  # Only display the top NO.15 companies
#     print(company, number)
    company, number = getSorted(company, number)
#     print(company, number)
    #============== Set up the Figures =================#
    of.offline.init_notebook_mode(connected=True)
    trace = [go.Bar(
        x = company, 
        y = number, 
        orientation='v'
    )]
    layout = go.Layout(plot_bgcolor='#E6E6FA', title =" Companies UCSD alumni went to", titlefont=dict(color='rgb(200, 100, 100)',size=30))
    data = [trace]
    fig = go.Figure(data = trace, layout = layout)
    of.iplot(fig)
    

{'1': 'University of California, Los Angeles', '3': 'Moebius Solutions, Inc.', '87': 'Amazon', '744': 'Others'}


### c. Industry that UCSD Alumni went to -- radar chart
#### reference links: https://plot.ly/python/radar-chart/

In [5]:
if __name__ == '__main__':
    #=============== Set up the data ===================#
    industries, number = seperateData(industry)
    print(industries)
    industries, number = getHighest(industries, number, 15)  # Only display the top NO.15 industries
    
    #============== Set up the Figures =================#
    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
      r=number,
      theta=industries,
      fill='toself',
      name='Industry that UCSD Alumni went to',
      opacity  = 0.5, # Set up the opacity of the figure
    ))
    
    fig.update_layout(
    polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 25]
    )),
    showlegend=True
        
)
    fig.show()

['Computer Software', 'Internet', 'Higher Education', 'Management Consulting', 'Information Technology & Services', 'Financial Services', 'Marketing & Advertising', 'Semiconductors', 'Investment Management', 'Consumer Electronics', 'Computer Hardware', 'Research', 'Investment Banking/Venture', 'Computer Networking', 'Electrical & Electronic Manufacturing', 'Computer & Network Security', 'Venture Capital', 'Wireless', 'Biotechnology', 'Chemicals', 'Banking', 'Program Development', 'Pharmaceuticals', 'Medical Device', 'Computer Games', 'Online Publishing', 'Mechanical or Industrial Engineering', 'Education Management', 'Telecommunications']


### d. Location that UCSD Alumni went to

In [47]:
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly
import geopandas

loc, value = seperate_Loc_Data(location)
loc, value = coordinate_Loc_Val(loc, value)
city, value = seperate_City_Data(location)
city_tuple, values = coordinate_City_Val(loc, city, value) # city is a tuple:(state, city)
fips = getFIPS(city_tuple, values)

scope = ['California']
values = range(len(fips))
colorscale = [
    'rgb(193, 193, 193)',
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(65, 53, 132)',
    'rgb(65, 32, 145)',
    'rgb(87, 96, 182)',
    'rgb(90, 10, 78)',
    'rgb(89, 32, 123)',
    'rgb(50, 140, 132)'
]

fig = ff.create_choropleth(fips=fips, values=values, scope=['CA'],county_outline={'color': 'rgb(155,155,255)', 'width': 1}, 
                           round_legend_values=True)
# fig = ff.create_choropleth(fips=fips, values=values, scope=scope, 
#                            colorscale=colorscale, round_legend_values=True,
#         simplify_county=0, simplify_state=0,     county_outline={'color': 'rgb(255, 255, 255)', 'width': 0.5},
#     state_outline={'width': 1},
#     legend_title='pop. per county',
#     title='California'                  
#                           )
fig.layout.template = None
fig.show()


{'725': '06073', '5': '06075', '17': '63000', '1': '17043', '4': '06086', '6': '06087'}


In [103]:
import plotly.graph_objects as go
import pandas as pd

# df = pd.read_csv('2011_us_ag_exports.csv')
loc, value = seperate_Loc_Data(location)
loc, value = coordinate_Loc_Val(loc, value)
line = plotly.graph_objects.choropleth.marker.Line()

fig = go.Figure(data=go.Choropleth(
    locations=loc, # Spatial coordinates
    z = value, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    marker_line_color = 'black', # display the color of the state boundary
    colorscale = 'Reds',
    colorbar_title = "Number of Alumni",
))
fig.update_layout(
    title_text = 'UCSD Alumni Employment Localization Distribution',
    geo_scope='usa', # limite map scope to USA
)
fig.show()
